In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS 
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
  temperature = 0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/chapter_one.txt")

docs = loader.load_and_split(text_splitter=splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are helpful assistant. Answer questions using onlythe following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
  ("human", "{question}")
])

chain = {"context":retriever,"question":RunnablePassthrough()} | prompt | llm

chain.invoke("Descrive Victory Mansion")

Failed to get info from http://api.smith.langchain.com: LangSmithConnectionError("Connection error caused failure to GET /info  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))")
Connection error caused failure to PATCH http://api.smith.langchain.com/runs/9860a6af-b288-4a7c-ab1d-a2d774970d74  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
Connection error caused failure to POST http://api.smith.langchain.com/runs  in LangSmith API. Please confirm your LANGCHAIN_ENDPOINT. ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


AIMessage(content='Victory Mansion is a building where Winston Smith resides. It is described as having glass doors that let in gritty dust, a hallway that smells of boiled cabbage and old rag mats, and a poster of a large face with the caption "BIG BROTHER IS WATCHING YOU." The flat where Winston lives is seven flights up, and the building has issues with the lift not working due to electricity cuts. The building is part of the city of London, in Airstrip One, which is one of the provinces of Oceania.')

In [2]:
#위 코드에서 벡터스토어에 배정된 문서의 벡터들을 벡터 공간에서 유사도 검색
vectorstore.similarity_search("where does winston")

[Document(page_content='Part 1, Chapter 1\nPart One\n1 It was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his breast in an effort to escape the vile wind, slipped quickly through the glass doors of Victory Mansions, though not quickly enough to prevent a swirl of gritty dust from entering along with him.\nThe hallway smelt of boiled cabbage and old rag mats. At one end of it a coloured poster, too large for indoor display, had been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of about forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was no use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut off during daylight hours. It was part of the economy drive in preparation for Hate Week. The flat was seven flights up, and Winston, who was thirty-nine and had a varico